<a href="https://colab.research.google.com/github/chrisrichardson/linear-algebra/blob/main/01_Linear_Algebra_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Import the libraries we are going to use
import numpy as np
import numba

# Matrices and vectors

* vector = 1D: e.g. `[1.0, 2.2, -3.0, 4.0]`
* matrix = 2D, e.g. $\left(\begin{matrix} 0.01 & 2.2 \\ -3.3 & 4.1 \end{matrix}\right)$
* tensor = N-dimensional

All of these are supported by `numpy`.


## Vectors

Note that a vector can be a row or a column, i.e. the shape can be `(1, N)` or `(N, 1)` - or just `(N,)`. The actual *value data* in memory is the same for all of these.

In [7]:
# Vector
b = np.array([1.0, 2.0, 3.0])
print(b, b.shape)
print()

# Row vector
b = np.array([[1.0, 2.0, 3.0]])
print(b, b.shape)
print()

# Column vector
b = np.array([[1.0], [2.0], [3.0]])
print(b, b.shape)
print()

[1. 2. 3.] (3,)

[[1. 2. 3.]] (1, 3)

[[1.]
 [2.]
 [3.]] (3, 1)



### dtype

A `numpy` array has a data type (or `dtype`) associated with it, which is usually a floating point type (e.g. 64-bit or "double" precision). You can set it or check it.

In [12]:
# Looking at dtypes
b = np.array([1.0, 2.0, 5.0, 8.0])
print(b, b.dtype)

b = np.array([1, 2, 3])
print(b, b.dtype)

b = np.array([1, 2, 3], dtype=np.float32)
print(b, b.dtype)


[1. 2. 5. 8.] float64
[1 2 3] int64
[1. 2. 3.] float32


### Operations on vectors

dot product: `a.dot(b)`

axpy: `w = a * x + y`

norm: `np.linalg.norm(a)`

In [23]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 0.5, -1.0])

# Dot product
print(a.dot(b))

# Norm
print(np.linalg.norm(a), np.linalg.norm(b))

# axpy
c = a + b
print(c)

# More unusual operation - pointwise multiplication:
c = a * b
print(c)


0.0
3.7416573867739413 2.29128784747792
[3.  2.5 2. ]
[ 2.  1. -3.]


## Matrices

A matrix is a two-dimensional array which can be used for linear algebra operations.

### Matrices in numpy

Matrices behave in numpy in a similar way to vectors.

**Storage**

The data is generally stored in contiguous memory, either *Row Major* (C-style) or *Column Major* (FORTRAN-style).
Every entry is stored (even if it is zero). This is called **dense** storage.

In [ ]:
A = np.array([[1, -2, 3],
              [3, 1, 5],
              [3, 4, 1]], dtype=np.float64)
print(A)
print(A.shape)
print(A.dtype)
print(A.flags)

[[ 1. -2.  3.]
 [ 3.  1.  5.]
 [ 3.  4.  1.]]
(3, 3)
float64
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [ ]:
@numba.njit
def matvec0(A, b):
  y = np.zeros(A.shape[0])
  for i in range(A.shape[0]):
    y[i] += np.dot(A[i, :], b[:])
  return y


In [ ]:
b = np.array([1, 2, 3], dtype=float)
print(b)
print(matvec0(A, b))

[1. 2. 3.]
[ 6. 20. 14.]


In [ ]:
@numba.njit
def matvec1(A, b):
  y = np.zeros(A.shape[0])
  for j in range(A.shape[1]):
    y += A[:, j] * b[j]
  return y

In [ ]:
print(matvec1(A, b))

[ 6. 20. 14.]


In [ ]:
print(A @ b)

[ 6. 20. 14.]


In [ ]:
print(np.dot(A, b))

[ 6. 20. 14.]


In [ ]:
print(np.matmul(A, b))

[ 6. 20. 14.]


Create a large matrix of random numbers (use `numpy.random.random`) and multiply it by a random vector b. Try the different matvec above, and see which is fastest.

In [ ]:
N = 1000
A = np.random.random((N, N))
# A = np.asfortranarray(A)
b = np.random.random(N)

In [ ]:
%%timeit -n 100
A @ b

904 µs ± 265 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -n 100
matvec0(A, b)

2.12 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -n 100
matvec1(A, b)

816 µs ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -n 100
np.dot(A, b)

1.63 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -n 100
np.matmul(A, b)

2.88 ms ± 583 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -n 100
np.einsum("ij,j->i", A, b)

3.63 ms ± 45.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
